In [10]:
def movie_list():
    req = urllib.request.Request(it, headers = headers)
    fhand = urllib.request.urlopen(req)
    soup = BeautifulSoup(fhand, 'html.parser') 
    
    ##extracting movie name with year##
    title = soup.find_all('div', attrs = {'class':'product_page_title oswald'})
    name = title[0].get_text().strip().replace('\n', ' ')
    movie_name.append(name)

In [11]:
def director_list():
    req = urllib.request.Request(it, headers = headers)
    fhand = urllib.request.urlopen(req)
    soup = BeautifulSoup(fhand, 'html.parser') 
    
    director_div = soup.find('div', attrs = {'class':'director'})
    director_link = director_div.find_all('a')
    #print(director_link)
    d=[]
    for i in director_link:
        d.append(i.get_text().strip())
    director_name.append(d) 

In [12]:
def genres_list():
    req = urllib.request.Request(it, headers = headers)
    fhand = urllib.request.urlopen(req)
    soup = BeautifulSoup(fhand, 'html.parser') 
    
    genres_div = soup.find('div', attrs = {'class':'genres'})
    genres_list = genres_div.find_all('span')
    genres = []
    for i in genres_list:
        genres.append(i.get_text())
    gen = genres[2:]
    genre_name.append(gen)

In [13]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import urllib.request, urllib.parse, urllib.error


headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}
all_links = []

for i in range(5):
    req = urllib.request.Request('https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc&page='+str(i), headers = headers)
    fhand = urllib.request.urlopen(req)
    soup = BeautifulSoup(fhand, 'html.parser') 
    hyperlinks = soup.find_all('a', attrs = {'class':'title'})
    all_links.extend(hyperlinks)
#len(all_links)
#print(all_links)


##extracting only urls from the links collected##
all_urls = []
for i in all_links:
    all_urls.append('https://www.metacritic.com' + i.get('href'))
#print(all_urls)

##all declarations##
movie_name = []
director_name = []
genre_name = []
dict500 = {}

for it in all_urls:
    movie_list()
    director_list()
    genres_list()

In [14]:
##creating the dictonary for 500 movies##
for i in range(len(all_urls)):
    sub_dict = {}
    sub_dict[tuple(director_name[i])] = genre_name[i]
    dict500[movie_name[i]] = sub_dict
    
#print(dict500)

In [15]:
##writing the dictionary into csv file##
import csv
fhandle = open('Joshi_movies.csv', mode='w', newline = '') #line 1
movie_writer = csv.writer(fhandle) #line 2
for key1, value1 in dict500.items():
    for key2, value2 in value1.items():            
        movie_writer.writerow([key1, ', '.join(list(key2)), ', '.join(value2)])
fhandle.close()

In [19]:
import pandas as pd
from scipy.spatial.distance import cosine

wish = 'Y'
while wish == 'Y' or wish == 'y': 
    inp = input("What do you want to check on Metacritic? (Please choose from 'Movies' , 'Director' or 'Comparison') \n")
    inp = inp.lower()
    if inp == 'movies':
        mov_sel = input("What movie do you want to check? \n")
        #mov_sel = mov_sel.capitalize()
        #print(mov_sel)
        m_director = []
        m_genres = []
        for key1, value1 in dict500.items():
            for key2, value2 in value1.items():
                if mov_sel in key1:
                    m_director.append(key2)
                    m_genres.extend(value2)

        print("The director of the movie is ",str(m_director)[2:-3])
        print("\nThe genre of the movie is ", str(m_genres)[1:-1])
        wish = input('Do you wish to continue? >>')
    

    elif inp == 'director':
        dir_sel = input("\nWho do you want to check? \n")
        d_movie = []
        d_genres = []
        for key1, value1 in dict500.items():
            for key2, value2 in value1.items():
                if dir_sel in key2:
                    d_movie.append(key1)
                    d_genres.extend(value2)

        d_g = {}
        for i in d_genres:
            if i not in d_g:
                d_g[i] = 1
            else:
                d_g[i] = d_g[i] + 1
        print("\n")       
        print(dir_sel, "has directed ",str(d_movie)[1:-1])
        print("His most directed genres are ", str(d_g)[1:-1])       
        wish = input('Do you wish to continue? >>')


    elif inp == 'comparison':
        dir1 = input("Who do you want to compare? \n 1> ")
        dir2 = input("\n 2>")

        d1_movie = []
        d1_genres = []
        d2_movie = []
        d2_genres = []   

        for key1, value1 in dict500.items():
            for key2, value2 in value1.items():
                if dir1 in key2:
                    d1_movie.append(key1)
                    d1_genres.extend(value2)

        d_g1 = {}
        for i in d1_genres:
            if i not in d_g1:
                d_g1[i] = 1
            else:
                d_g1[i] = d_g1[i] + 1

        for key1, value1 in dict500.items():
            for key2, value2 in value1.items():
                if dir2 in key2:
                    d2_movie.append(key1)
                    d2_genres.extend(value2)

        d_g2 = {}
        for i in d2_genres:
            if i not in d_g2:
                d_g2[i] = 1
            else:
                d_g2[i] = d_g2[i] + 1

        common_genres = set(d1_genres).union(set(d2_genres))


        dict_common1 = {}
        for i in common_genres:
            dict_common1[i] = 0
        dict_common1.update(d_g1)
        #dict_common1

        dict_common2 = {}
        for i in common_genres:
            dict_common2[i] = 0
        dict_common2.update(d_g2)
        #dict_common2

        dict_c1s = pd.Series(dict_common1)
        dict_c2s = pd.Series(dict_common2)
        score = (1 - cosine(dict_c1s, dict_c2s))

        print(dir1, "has directed ",str(d1_movie)[1:-1])
        print("His most directed genres are ", str(d_g1)[1:-1])
        print("\n")
        print("\n",dir2, "has directed ",str(d2_movie)[1:-1])
        print("His most directed genres are ", str(d_g2)[1:-1])
        print("\n")
        print("\n Based on that they have cosine similarity score of " , score)
        #print(dict_common1)
        #print(dict_common2)
        wish = input('Do you wish to continue? >> ')
        
    else:
        print("\nYou entered wrong choice. \n")
        continue

What do you want to check on Metacritic? (Please choose from 'Movies' , 'Director' or 'Comparison') 
movies
What movie do you want to check? 
One False Move 1992
The director of the movie is  'Carl Franklin'

The genre of the movie is  'Drama', 'Crime'
Do you wish to continue? >>y
What do you want to check on Metacritic? (Please choose from 'Movies' , 'Director' or 'Comparison') 
movies
What movie do you want to check? 
F for Fake 1975
The director of the movie is  'François Reichenbach', 'Gary Graver', 'Oja Kodar', 'Orson Welles

The genre of the movie is  'Documentary'
Do you wish to continue? >>y
What do you want to check on Metacritic? (Please choose from 'Movies' , 'Director' or 'Comparison') 
movies
What movie do you want to check? 
The Godfather 1972
The director of the movie is  'Francis Ford Coppola'

The genre of the movie is  'Drama', 'Thriller', 'Crime'
Do you wish to continue? >>y
What do you want to check on Metacritic? (Please choose from 'Movies' , 'Director' or 'Compar

In [ ]:
len(dict500)